## `RNN`
***
***
Time: 2020-09-13<br>
Author: dsy
***

$
\begin{aligned}
z_t & = Uh_{t-1} + Wx_t+b \\
h_t & = f(z_t) \\
y_t & = Vh_t \\
x_t \text{是网络的输入}，h_t \text{隐藏层状态}，z_t \text{隐藏层的净输入}，& f(\cdot) \text{
是非线性激活函数，通常为Logistic函数或者Tanh函数}，U \text{状态-状态权重矩阵},W \text{状态-输入权重矩阵},b\text{为偏置}
\end{aligned}
$

In [1]:
import torch
import torch.nn as nn

In [2]:
#torch.manual_seed(0)

In [3]:
class RNNFromDsy2(nn.Module):
    def __init__(self):
        super(RNNFromDsy2,self).__init__()
        
    def forward(self,x):
        m,n = x.shape
        U = torch.rand(1,requires_grad=True)
        W = torch.rand(1,requires_grad=True)
        b = torch.rand(1,requires_grad=True)
        V = torch.rand(1,requires_grad=True)
        y = torch.empty((m,n),dtype=torch.float32)
        ht = torch.zeros((m,n+1),dtype=torch.float32)
        
        for i in range(m):
            for j in range(n):
                ht[i][j+1] = torch.sigmoid(V*( U*(ht[i][j]) + W*(x[i][j]) + b))
                y[i][j] = V * (ht[i][j+1])
        return y,ht[:,1:]
        

In [4]:
X = torch.randn((4,6))
X

tensor([[-0.6920, -0.7670, -0.5207,  0.9815, -0.0420,  0.1500],
        [ 0.3686, -1.4934, -0.1194,  2.0738,  0.1585,  0.3091],
        [ 0.5719,  0.4486,  0.1359,  0.3469,  0.8337, -0.0827],
        [ 1.9388,  0.6318, -0.4251, -1.2832,  0.9672, -0.0168]])

In [5]:
rnnfd2 = RNNFromDsy2()
y,ht = rnnfd2(X)

In [6]:
y

tensor([[0.5082, 0.5503, 0.5697, 0.6599, 0.6086, 0.6157],
        [0.5759, 0.5098, 0.5909, 0.7173, 0.6253, 0.6264],
        [0.5885, 0.6315, 0.6168, 0.6279, 0.6564, 0.6059],
        [0.6675, 0.6483, 0.5844, 0.5242, 0.6556, 0.6098]],
       grad_fn=<CopySlices>)

In [7]:
ht

tensor([[0.5411, 0.5860, 0.6066, 0.7026, 0.6480, 0.6555],
        [0.6132, 0.5428, 0.6292, 0.7637, 0.6658, 0.6670],
        [0.6266, 0.6723, 0.6567, 0.6686, 0.6989, 0.6451],
        [0.7108, 0.6903, 0.6222, 0.5582, 0.6980, 0.6492]],
       grad_fn=<SliceBackward>)

In [8]:
# 以前的实现想法
class RNNFromDsy(nn.Module):
    def __init__(self,hidden_size=10):
        super(RNNFromDsy,self).__init__()
        self.hidden_size = hidden_size
        
    def forward(self,x):
        self.m,self.n = x.shape
        ht = torch.zeros((self.batch,self.input_size))
        U = torch.empty((self.batch,self.batch),requires_grad=True)
        W = torch.empty((self.batch,self.batch),requires_grad=True)
        b = torch.empty((self.batch,self.input_size),requires_grad=True)
        V = torch.empty((self.batch,self.batch),requires_grad=True)
        nn.init.uniform_(U)
        nn.init.uniform_(W)
        nn.init.uniform_(b)
        nn.init.uniform_(V)
        for i in range(self.hidden_size):
            ht = torch.tanh( U.matmul(ht) + W.matmul( x) + b)
        return V.matmul(ht)